In [227]:
import pandas as pd
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

rng = np.random.default_rng()
flow_matrix_path = "../recovery_rate/recovery_rate_training_dataframes/data/all_time_flight_flow_df_rev1.pickle"
flight_flow_data = pd.read_pickle(flow_matrix_path)

infection_matrix_path = "../recovery_rate/recovery_rate_training_dataframes/data/all_time_infection_rate_df_rev1.pickle"
infection_matrix_df = pd.read_pickle(infection_matrix_path)

apt_df_path = "../recovery_rate/recovery_rate_training_dataframes/data/airport_state_weather_prediction_added_rev1.pickle"
airport_df = pd.read_pickle(apt_df_path)

mult_index = pd.MultiIndex.from_product([np.arange(np.datetime64("2018-01-01"), np.datetime64("2018-07-01")), range(0, 96, 4)], names=["date", "tw"])

new_df = pd.DataFrame(index=mult_index)


In [228]:
ff_list = []
inf_list = []
rr_list = []
p_state_list = []
for i in range(0, len(flight_flow_data), 133):
    
    rr_list.append(np.diag(airport_df.loc[:, "recovery_rate"].fillna(method="backfill").fillna(airport_df.loc[:, "recovery_rate"].median()).iloc[i:i+133].values))
    p_state_list.append(airport_df.loc[:, "norm_delay_per_f"].iloc[i:i+133].values.clip(min=0, max=1))
    ff_list.append(flight_flow_data.iloc[i:i+133, :].values)
    inf_list.append(infection_matrix_df.iloc[i:i+133, :].values)

In [229]:
new_df["rr_vector"] = rr_list
new_df["p_vector"] = p_state_list
new_df["infmat"] = inf_list
new_df["flowmat"] = ff_list

In [213]:
type(slice_df.apply(diff_prop, axis=1))

pandas.core.series.Series

## Delta P plots


In [1]:
random = True
if random:
    date = str(rng.choice(np.arange(np.datetime64("2018-01-01"), np.datetime64("2018-07-01"))))
    tw=slice(24, 72, 1)
    apt_idx = rng.integers(0, 133)
else:
    date="2018-04-05"
    tw=slice(24, 72, 1)
    apt_idx = 5
    
index = (date, tw)

apt_name = airport_df.index.unique("apt").tolist()[apt_idx]

def diff_prop(slice_df):
    return (((slice_df["infmat"] - slice_df["rr_vector"]) @ slice_df["p_vector"]) - ((slice_df["infmat"] @ slice_df["p_vector"]) * slice_df["p_vector"])) / 1

slice_df = deepcopy(new_df.loc[index, :])

def apt_index(vector):
    return vector[apt_idx]

plot_kwargs = dict(figsize=(12, 6), xlabel="tw", ylabel="P Delta", legend=True, xticks=range(24, 72, 4), title=f"Change of 'delay per flight per hour' for {apt_name}")


slice_df["calc_p"] = slice_df.apply(diff_prop, axis=1).values
slice_df["calc_p"].reset_index(level="date", drop=True).apply(apt_index).plot(**plot_kwargs, label="Calculated P")
slice_df["p_vector"].reset_index(level="date", drop=True).apply(apt_index).diff().shift(-1).plot(**plot_kwargs, label="Actual P")
plt.show()

NameError: name 'rng' is not defined

## Delta P Plots with Uncertainties

In [ ]:
uncertainty_matrix = rng.normal(0,) 